In [ ]:
!pip install kaggle

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/Kaggle/carvana/carvana-image-masking-challenge"))

import zipfile
import shutil

DATASET_DIR = '/content/drive/MyDrive/Kaggle/carvana/carvana-image-masking-challenge/'
WORKING_DIR = '/content/drive/MyDrive/Kaggle/carvana/Working/'

['metadata.csv.zip', '29bb3ece3180_11.jpg', 'sample_submission.csv.zip', 'test.zip', 'test_hq.zip', 'train.zip', 'train_hq.zip', 'train_masks.csv.zip', 'train_masks.zip']


In [ ]:
if len(os.listdir(WORKING_DIR)) <= 1:

    with zipfile.ZipFile(DATASET_DIR + 'train.zip', 'r') as zip_file:
        zip_file.extractall(WORKING_DIR)

    with zipfile.ZipFile(DATASET_DIR + 'train_masks.zip', 'r') as zip_file:
        zip_file.extractall(WORKING_DIR)

    print(
        len(os.listdir(WORKING_DIR + 'train')),
        len(os.listdir(WORKING_DIR + 'train_masks'))
    )

    train_dir = WORKING_DIR + 'train/'
    val_dir = WORKING_DIR + 'val/'
    os.mkdir(val_dir)
    for file in sorted(os.listdir(train_dir))[4600:]:
      shutil.move(train_dir + file, val_dir)

    masks_dir = WORKING_DIR + 'train_masks/'
    val_masks_dir = WORKING_DIR + 'val_masks/'
    os.mkdir(val_masks_dir)
    for file in sorted(os.listdir(masks_dir))[4600:]:
      shutil.move(masks_dir + file, val_masks_dir)

    os.mkdir(WORKING_DIR + 'saved_images')

In [ ]:
sorted(os.listdir(WORKING_DIR + 'train'))[:10]

['00087a6bd4dc_01.jpg',
 '00087a6bd4dc_02.jpg',
 '00087a6bd4dc_03.jpg',
 '00087a6bd4dc_04.jpg',
 '00087a6bd4dc_05.jpg',
 '00087a6bd4dc_06.jpg',
 '00087a6bd4dc_07.jpg',
 '00087a6bd4dc_08.jpg',
 '00087a6bd4dc_09.jpg',
 '00087a6bd4dc_10.jpg']

In [ ]:
sorted(os.listdir(WORKING_DIR + 'train_masks'))[:10]

['00087a6bd4dc_01_mask.gif',
 '00087a6bd4dc_02_mask.gif',
 '00087a6bd4dc_03_mask.gif',
 '00087a6bd4dc_04_mask.gif',
 '00087a6bd4dc_05_mask.gif',
 '00087a6bd4dc_06_mask.gif',
 '00087a6bd4dc_07_mask.gif',
 '00087a6bd4dc_08_mask.gif',
 '00087a6bd4dc_09_mask.gif',
 '00087a6bd4dc_10_mask.gif']

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class CarvanaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace('.jpg', '_mask.gif'))
        image = np.array(Image.open(img_path).convert('RGB'))
        mask = np.array(Image.open(mask_path).convert('L'), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations['image']
            mask = augmentations['mask']

        return image, mask

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64,128,256,512]):
        super(UNET, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2,feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], 1, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:], antialias=None)

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

"""
def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(1, 1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)

    assert preds.shape == x.shape

if __name__ == "__main__":
    test()

"""

torch.Size([3, 1, 161, 161])
torch.Size([3, 1, 161, 161])


In [ ]:
import torch
import torchvision
# from dataset import CarvanaDataset
from torch.utils.data import DataLoader

def save_checkpoint(state, filename='my_checkpoint.pth.tar'):
    print('=> Saving checkpoint')
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print('=> Loading checkpoint')
    model.load_state_dict(checkpoint['state_dict'])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CarvanaDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True
    )

    val_ds = CarvanaDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False
    )

    return train_loader, val_loader

def check_accuracy(loader, model, device='cuda'):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

    print(
        f'Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}'
    )
    print(f'Dice score: {dice_score/len(loader)}')
    model.train()
    return float(num_correct) / float(num_pixels) * 100

def save_predictions_as_imgs(
        loader, model, folder=WORKING_DIR+'saved_images', device='cuda'
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f'{folder}/pred_{idx}.png'
        )
        torchvision.utils.save_image(
            y.unsqueeze(1), f'{folder}/truth_{idx}.png'
        )

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from PIL import Image, UnidentifiedImageError
# from model import UNET
# from utils import (
#     load_checkpoint,
#     save_checkpoint,
#     get_loaders,
#     check_accuracy,
#     save_predictions_as_imgs,
# )

# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 16
NUM_EPOCHS = 5
NUM_WORKERS = 2
IMAGE_HEIGHT = 320
IMAGE_WIDTH = 480
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = '/content/drive/MyDrive/Kaggle/carvana/Working/train'
TRAIN_MASK_DIR = '/content/drive/MyDrive/Kaggle/carvana/Working/train_masks'
VAL_IMG_DIR = '/content/drive/MyDrive/Kaggle/carvana/Working/val'
VAL_MASK_DIR = '/content/drive/MyDrive/Kaggle/carvana/Working/val_masks'

def delete_corrupted_files(corrupted_files):
    # corrupted_files should be a list of tuples where each tuple contains (image_path, mask_path)
    for img_path, mask_path in corrupted_files:
        if os.path.exists(img_path):
            print(f"Deleting corrupted image: {img_path}")
            os.remove(img_path)
        if os.path.exists(mask_path):
            print(f"Deleting corrupted mask: {mask_path}")
            os.remove(mask_path)

# Example usage:
# Assuming you already have a list of corrupted image and mask paths
corrupted_files = [
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_02.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_02_mask.gif"),
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_04.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_04_mask.gif"),
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_06.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_06_mask.gif"),
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_11.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_11_mask.gif"),
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_12.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_12_mask.gif"),
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_13.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_13_mask.gif"),
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_14.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_14_mask.gif"),
    ("/content/drive/MyDrive/Kaggle/carvana/Working/train/c8d52e0ba341_15.jpg", "/content/drive/MyDrive/Kaggle/carvana/Working/train_masks/c8d52e0ba341_15_mask.gif")
    # Add more corrupted file pairs
]

delete_corrupted_files(corrupted_files)

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
      data = data.to(device=DEVICE)
      targets = targets.float().unsqueeze(1).to(device=DEVICE)

      # forward
      with torch.cuda.amp.autocast():
          predictions = model(data)
          loss = loss_fn(predictions, targets)

      # backward
      optimizer.zero_grad()
      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update()

      # update tqdm loop
      loop.set_postfix(loss=loss.item())

def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ]
    )

    val_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ]
    )

    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transform,
        NUM_WORKERS,
        PIN_MEMORY
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load('my_checkpoint.pth.tar'), model)
        check_accuracy(val_loader, model, device=DEVICE)

    scaler = torch.cuda.amp.GradScaler()

    best_acc = 0.0  # Track the best accuracy

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # Evaluate model on validation set
        acc = check_accuracy(val_loader, model, device=DEVICE)

        # Save best model if accuracy improves
        if acc > best_acc:
            best_acc = acc
            checkpoint = {
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }
            torch.save(checkpoint, "/content/drive/MyDrive/Kaggle/carvana/best_model.pth.tar")

        # Optionally save predictions every epoch
        save_predictions_as_imgs(
            val_loader, model, folder=WORKING_DIR + 'saved_images', device=DEVICE
        )

if __name__ == '__main__':
    main()

<ipython-input-14-9902d0f75a21>:127: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/287 [00:00<?, ?it/s]<ipython-input-14-9902d0f75a21>:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 287/287 [05:00<00:00,  1.05s/it, loss=0.137]


Got 73238308/74956800 with acc 97.71
Dice score: 0.9491408467292786


100%|██████████| 287/287 [04:20<00:00,  1.10it/s, loss=0.0638]


Got 72790650/74956800 with acc 97.11
Dice score: 0.9358701705932617


100%|██████████| 287/287 [04:21<00:00,  1.10it/s, loss=0.0379]


Got 72511926/74956800 with acc 96.74
Dice score: 0.92940753698349


100%|██████████| 287/287 [04:18<00:00,  1.11it/s, loss=0.0292]


Got 73687845/74956800 with acc 98.31
Dice score: 0.9620025753974915


100%|██████████| 287/287 [04:20<00:00,  1.10it/s, loss=0.0217]


Got 73668635/74956800 with acc 98.28
Dice score: 0.9613279700279236
